# Azure OpenAI Service Embeddings 기초
임베딩(Embeddings)은 머신러닝 분야에서 사용하는 특별한 데이터 타입이다. 임베딩은 텍스트가 가진 의미를 고밀도의 정보로 표기한다. 각각의 임베딩은 부동 소수점 벡터이며, 벡터 공간에서 2개의 임베딩 사이의 거리는 임베딩의 원본인 텍스트 간 의미적 유사성과 연관이 있다. 이처럼 벡터를 사용하면 2개의 텍스트가 얼마나 비슷한지 계산할 수 있다.

## 사전 준비

이 파이썬 예제를 실행하려면 다음과 같은 환경이 필요하다:

- Azure OpenAI Service를 사용할 수 있는 [승인 완료](https://aka.ms/oai/access)된 Azure 구독
- Azure OpenAI Service에 배포된 Embeddings 모델
- Azure OpenAI Service 연동 및 모델 정보
  - OpenAI API 키
  - OpenAI Embeddings 모델의 배포 이름. 여기서는 기본값인 `text-embedding-ada-002`를 사용한다.
  - OpenAI API 버전
- Python (이 예제는 버전 3.10.x로 테스트 했다.)

이 예제에서는 Visual Studio Code와 [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)를 사용한다.


## 패키지 설치


In [ ]:
!pip install openai

In [ ]:
import openai
openai.__version__

## 라이브러리 및 환경변수 불러오기


## Azure OpenAI 설정
Azure OpenAI와 연동을 위해 필요한 정보는 보안을 위해 하드코딩 하지 말고 환경변수나 [dotenv](https://pypi.org/project/python-dotenv/)로 불러오는 것을 권장한다.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

#os.environ["AZURE_OPENAI_API_KEY"] = "Your OpenAI API Key"
#os.environ["AZURE_OPENAI_ENDPOINT"] = "https://<Your OpenAI Service>.openai.azure.com/"

# 이 변수에는 모델을 배포했을 때 설정한 커스텀 이름을 입력한다.
AZURE_OPENAI_DEPLOYMENT_NAME = "text-embedding-ada-002"

## 벡터 간 거리 계산하기
벡터 간 거리를 계산할 때는 코사인 유사도를 주로 사용한다. 다음은 2개의 벡터 간 코사인 유사도를 계산하는 코드다.

In [ ]:
# 필요한 라이브러리를 설치한다.
#!pip install numpy matplotlib scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calc_cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    cos = dot_product / (norm_v1 * norm_v2)
    return cos

v1 = np.array([1, 0])
v2 = np.array([1, 1])

cosine_similarity = calc_cosine_similarity(v1, v2)

print("코사인 유사도: ", cosine_similarity)
print("코사인 거리: ", 1 - cosine_similarity)

In [ ]:
def plot_vectors(vectors, labels):
    """
    지정한 벡터를 2차원 그래프에 나타내고, 레이블을 표시하는 함수 
    
    Parameters:
    vectors (list of numpy.ndarray): 그래프에 나타낼 벡터 목록
    labels (list of str): 벡터 레이블 목록
    """
    # 그래프 설정
    plt.figure()
    colors = ['b', 'r', 'g', 'c', 'm', 'y']  # 벡터마다 다른 색을 사용한다.
    for i, vector in enumerate(vectors):
        plt.quiver(0, 0, vector[0], vector[1], angles='xy', scale_units='xy', scale=1, color=colors[i], label=labels[i])
        plt.text(vector[0] + 0.1, vector[1] + 0.1, f'{labels[i]} ({vector[0]}, {vector[1]})', color=colors[i], fontsize=12)

    # 그래프 설정(정사각형으로 조정)
    plt.axis("square")

    # 그래프 설정(레이블, 축의 범위 등)
    plt.xlim(-2, 2)
    plt.ylim(-2, 2)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.axhline(0, color='black',linewidth=0.5)
    plt.axvline(0, color='black',linewidth=0.5)
    plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5)
    plt.legend()

    # 그래프 표시
    plt.show()

In [ ]:
# 벡터 정의
v1 = np.array([1, 0])
v2 = np.array([1, 1])

# 벡터 레이블
labels = ['v1', 'v2']

# 함수를 호출하여 벡터 그리기
plot_vectors([v1, v2], labels)

## Embeddings API 사용하기

텍스트를 임베딩 모델에 전달하면 텍스트는 일련의 부동소수점으로 변환된다. 이 변환된 값을 로컬에 저장하거나 애저의 데이터 저장소에 저장해서 벡터 검색을 확장할 수 있다.

In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2024-02-01",
  azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

def generate_embeddings(text, model=AZURE_OPENAI_DEPLOYMENT_NAME): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding


In [ ]:
text_1 = "저는 노르웨이 숲 고양이가 좋아요."
text_2 = "저는 러시안 블루 고양이를 좋아해요."

embeddings_1 = generate_embeddings(text_1)
embeddings_2 = generate_embeddings(text_2)

embeddings_1

### 2개의 벡터 간 코사인 유사도 계산하기
코사인 유사도는 1에 가까울수록 유사한 의미를 가진다.

In [ ]:
print(calc_cosine_similarity(embeddings_1, embeddings_2))

In [ ]:
print(calc_cosine_similarity(embeddings_1, embeddings_1))

In [ ]:
text_3 = "저는 푸들이 좋아요."

embeddings_3 = generate_embeddings(text_3)
print(calc_cosine_similarity(embeddings_1, embeddings_3))

In [ ]:
text_4 = "저는 Azure AI Search를 사용해서 RAG 시스템을 만들었어요."

embeddings_4 = generate_embeddings(text_4)
print(calc_cosine_similarity(embeddings_3, embeddings_4))

## K-means를 사용한 군집화
K-means를 사용하는 가장 간단한 방법을 살펴보자. 군집화는 데이터 내부에 숨겨진 중요한 그룹들을 발견하는 데 도움이 된다.

In [ ]:
from sklearn.cluster import KMeans
# K-Means 군집화 실행
# 군집의 수는 간단한 예시를 만들기 위해 3으로 설정했다. 실제로는 데이터에 따라 적절하게 변경해야 한다.
kmeans = KMeans(n_clusters=3, random_state=0)
data = np.array([embeddings_1, embeddings_2, embeddings_3, embeddings_4])
clusters = kmeans.fit_predict(data)
clusters

### Embeddings 시각화(군집화)
1,536 차원의 벡터를 가진 데이터를 2차원 좌표상에 나타낼 때는 차원축소를 사용하는 것이 좋다. 단, 여기서는 예시 데이터의 수(4개)가 매우 적기 때문에 제한된 결과만 제공한다는 점에 유의해야 한다. 아래 링크를 참고하여 예시 데이터 수를 늘려서 데이터의 전체적인 모습을 파악해보길 바란다.

In [ ]:
from sklearn.decomposition import PCA

def plot_pca_2d(data, labels, clusters):
    """
    2차원 PCA를 사용해서 데이터를 나타내는 함수
    
    :param data: 2차원으로 축소할 고차원 데이터(numpy 배열)
    :param labels: 각 데이터 포인트에 해당하는 레이블 목록
    :param clusters: 각 데이터 포인트의 군집 할당(정수 배열)
    """
    if len(data) != len(labels):
        raise ValueError("데이터 포인트 수와 레이블 수가 일치하지 않습니다.")

    # PCA를 사용해서 2차원으로 축소
    pca = PCA(n_components=2)
    reduced_data = pca.fit_transform(data)

    # 결과 시각화(군집별로 다른 색상 사용)
    plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=clusters)
    for i, (x, y) in enumerate(reduced_data):
        plt.text(x, y, labels[i], fontsize=9)

    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('2D PCA Visualization with Clusters')
    plt.show()

In [ ]:
data = np.array([embeddings_1, embeddings_2, embeddings_3, embeddings_4])
labels = ['Norwegian Forest', 'Scottish Fold', 'Poodle', 'Azure AI Search']
plot_pca_2d(data, labels, clusters)

대량의 데이터셋을 사용한 Embeddings 변환 및 벡터 검색은 [Docs](https://learn.microsoft.com/azure/ai-services/openai/tutorials/embeddings?tabs=python-new%2Ccommand-line&pivots=programming-language-python)나 [00_DataIngest_AzureAISearch_PythonSDK.ipynb](../00_DataIngest_AzureAISearch_PythonSDK.ipynb)을 참고하자.